In [1]:
from utils.data_preparation import get_emotion_dataset, naive_bayes_preprocessing
import pandas as pd

/Users/rishabhthaney/.pyenv/versions/3.11.6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = get_emotion_dataset()
train_dataset = dataset['train']
df_train = pd.DataFrame(train_dataset)

### Data Fields

- text: a string feature
- label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [4]:
# print the first 5 rows of the dataset
df_train.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [6]:
# different labels
df_train['label'].unique()

array([0, 3, 2, 5, 4, 1])

In [7]:
# average sentence length
df_train['text'].str.split().str.len().mean()

19.1663125

### Preprocessing

In [12]:
processed_data = naive_bayes_preprocessing(
    remove_stopwords=True, 
    use_bigrams=False
)
print(processed_data[0]['train'])
# X_train, y_train = processed_data['train']
# X_val, y_val = processed_data['validation']
# X_test, y_test = processed_data['test']

{'train': (<16000x14894 sparse matrix of type '<class 'numpy.int64'>'
	with 130517 stored elements in Compressed Sparse Row format>, array([0, 0, 3, ..., 1, 3, 0])), 'validation': (<2000x14894 sparse matrix of type '<class 'numpy.int64'>'
	with 15161 stored elements in Compressed Sparse Row format>, array([0, 0, 2, ..., 1, 1, 1])), 'test': (<2000x14894 sparse matrix of type '<class 'numpy.int64'>'
	with 15261 stored elements in Compressed Sparse Row format>, array([0, 0, 0, ..., 1, 1, 4]))}


In [9]:
# print shapes of the data
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

NameError: name 'X_train' is not defined

In [10]:
# print first row of the training data
# NOTE: the data represents the non-zero indices of the sparse matrix
print(X_train[0])


NameError: name 'X_train' is not defined